<a href="https://colab.research.google.com/github/amien1410/amien-scrapers/blob/main/PoysoMotor_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [19]:
def safe_get_text(parent, selector, attr=None):
    try:
        el = parent.select_one(selector)
        if not el:
            return ""
        if attr:
            return el.get(attr, "").strip()
        return el.text.strip()
    except Exception:
        return ""

def extract_car_data(car_div):
    try:
        stock_no = car_div.get("data-stockno", "")
        vin = car_div.get("data-vin", "")

        # Safely get images
        try:
            images = [img.get("src", "").strip() for img in car_div.select(".embla__slide img") if img.get("src")]
        except Exception:
            images = []

        # Safely get badges
        try:
            badges = [badge.text.strip() for badge in car_div.select(".banner-badge") if badge]
        except Exception:
            badges = []

        # Title and basic info
        title_link = car_div.select_one(".si-title")
        year = safe_get_text(title_link, ".year") if title_link else ""
        make = safe_get_text(title_link, ".make") if title_link else ""
        model = safe_get_text(title_link, ".model") if title_link else ""
        badge_detail = safe_get_text(title_link, ".badge") if title_link else ""
        full_title = title_link.get("title", "").strip() if title_link and title_link.has_attr("title") else ""

        # Price
        price_value = safe_get_text(car_div, ".si-price .price-value")
        price_text = safe_get_text(car_div, ".si-price .price-text")

        # Features
        features = car_div.select_one(".si-features")
        category = safe_get_text(features, ".sif-category") if features else ""
        odometer = safe_get_text(features, ".odometer") if features else ""
        consumption = safe_get_text(features, ".consumption") if features else ""
        body_type = safe_get_text(features, ".body-type") if features else ""
        fuel = safe_get_text(features, ".fuel") if features else ""

        # Location
        location = safe_get_text(car_div, ".si-location span")

        return {
            "stock_no": stock_no,
            "vin": vin,
            "images": images,
            "badges": badges,
            "year": year,
            "make": make,
            "model": model,
            "badge_detail": badge_detail,
            "full_title": full_title,
            "price_value": price_value,
            "price_text": price_text,
            "category": category,
            "odometer": odometer,
            "consumption": consumption,
            "body_type": body_type,
            "fuel": fuel,
            "location": location
        }

    except Exception as e:
        print(f"Error parsing car_div: {e}")
        return None  # or return {} depending on your use case


# Setup
url = "https://www.poysermotorgroup.com.au/stock/results/load"
all_cars = []
page = 1

while True:
    payload = {
        "page": page
    }

    res = requests.post(url, data=payload)
    html = res.json().get("html", "")

    if not html.strip():
        break  # Stop if HTML is empty

    soup = BeautifulSoup(html, "html.parser")
    car_divs = soup.select("div.stock-item")

    for car_div in car_divs:
        car_data = extract_car_data(car_div)
        all_cars.append(car_data)

    print(f"Scraped page {page}, found {len(car_divs)} cars.")

    if len(car_divs) < 24:
        break

    page += 1

Scraped page 1, found 24 cars.
Scraped page 2, found 24 cars.
Scraped page 3, found 24 cars.
Scraped page 4, found 24 cars.
Scraped page 5, found 24 cars.
Scraped page 6, found 24 cars.
Scraped page 7, found 24 cars.
Scraped page 8, found 24 cars.


In [20]:
# Output sample or full list
# import pprint
# pprint.pprint(all_cars)

df = pd.DataFrame(all_cars)
df

,stock_no,vin,images,badges,year,make,model,badge_detail,full_title,price_value,price_text,category,odometer,consumption,body_type,fuel,location
0,507723,5N1MDAR53A0404042,[https://edge.pxcrush.net/dealerweb/car/cil/j9...,"[Added 2 days ago, FREE TOW PACK INCLUDED IN J...",2024,Nissan,Pathfinder,Ti R53 Auto 2WD MY24,2024 Nissan Pathfinder Ti R53 Auto 2WD MY24,"$69,980",Drive Away,Demo,26 km,10.0L / 100km,SUV,3.5L Petrol,Bendigo
1,703622,MPBUMFE60MX359862,[https://edge.pxcrush.net/cars/dealer/6ej42wgk...,[Added 3 days ago],2021,Ford,Ranger,Raptor X PX MkIII Auto 4x4 MY21.75 Double Cab,2021 Ford Ranger Raptor X PX MkIII Auto 4x4 MY...,"$54,990",Drive Away,Used,"98,089 km",8.2L / 100km,Ute,2.0L Diesel,Bendigo
2,507686,LGWEEUA56SL623558,[https://edge.pxcrush.net/dealerweb/car/cil/xl...,[Added 4 days ago],2025,GWM,Haval Jolion,Ultra Hybrid Auto,2025 GWM Haval Jolion Ultra Hybrid Auto,"$39,485",Drive Away,Demo,14 km,5.1L / 100km,SUV,1.5L Petrol,Epsom
3,703627,KMHK381AMKU366359,[https://edge.pxcrush.net/cars/dealer/crrxmi57...,[Added 4 days ago],2019,Hyundai,Kona,Elite Auto 2WD MY19,2019 Hyundai Kona Elite Auto 2WD MY19,"$25,990",Drive Away,Used,"72,119 km",7.2L / 100km,SUV,2.0L Petrol,Epsom
4,507653,LGWEF6A51SH972803,[https://edge.pxcrush.net/cars/dealer/b6fwkfhq...,[],2025,GWM,Haval H6,Vanta Auto,2025 GWM Haval H6 Vanta Auto,"$36,990",Drive Away,New,13 km,7.4L / 100km,SUV,2.0L Petrol,Epsom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,507265,KMHJC81BWSU377542,[https://media.adtorqueedge.com/stock/images/j...,[],2025,Hyundai,Tucson,Premium Auto AWD MY25,2025 Hyundai Tucson Premium Auto AWD MY25,"$57,500",Drive Away,New,11 km,7.2L / 100km,SUV,1.6L Petrol,Epsom
192,507263,KMHJB811MSU332478,[https://media.adtorqueedge.com/stock/images/j...,[On Special],2025,Hyundai,Tucson,Elite Auto 2WD MY25,2025 Hyundai Tucson Elite Auto 2WD MY25,"$50,000",Drive Away,New,8 km,5.3L / 100km,SUV,1.6L Petrol,Epsom
193,506948,3GCUD9EL3RG345545,[https://media.adtorqueedge.com/stock/images/j...,[],2024,Chevrolet,Silverado,1500 ZR2 W/Tech Pack Auto 4x4 MY24,2024 Chevrolet Silverado 1500 ZR2 W/Tech Pack ...,"$139,888",Drive Away,Demo,"4,685 km",12.2L / 100km,Ute,6.2L Petrol,Epsom
194,507321,WVGZZZA18SV050512,[https://media.adtorqueedge.com/stock/images/j...,[],2025,Volkswagen,T-Roc,110TSI Style D11 Auto MY25,2025 Volkswagen T-Roc 110TSI Style D11 Auto MY25,POA,Price on Application,New,12 km,6.3L / 100km,SUV,1.4L Petrol,Epsom


In [21]:
df.drop_duplicates(subset=["stock_no"], inplace=True)
display(df)

,stock_no,vin,images,badges,year,make,model,badge_detail,full_title,price_value,price_text,category,odometer,consumption,body_type,fuel,location
0,507723,5N1MDAR53A0404042,[https://edge.pxcrush.net/dealerweb/car/cil/j9...,"[Added 2 days ago, FREE TOW PACK INCLUDED IN J...",2024,Nissan,Pathfinder,Ti R53 Auto 2WD MY24,2024 Nissan Pathfinder Ti R53 Auto 2WD MY24,"$69,980",Drive Away,Demo,26 km,10.0L / 100km,SUV,3.5L Petrol,Bendigo
1,703622,MPBUMFE60MX359862,[https://edge.pxcrush.net/cars/dealer/6ej42wgk...,[Added 3 days ago],2021,Ford,Ranger,Raptor X PX MkIII Auto 4x4 MY21.75 Double Cab,2021 Ford Ranger Raptor X PX MkIII Auto 4x4 MY...,"$54,990",Drive Away,Used,"98,089 km",8.2L / 100km,Ute,2.0L Diesel,Bendigo
2,507686,LGWEEUA56SL623558,[https://edge.pxcrush.net/dealerweb/car/cil/xl...,[Added 4 days ago],2025,GWM,Haval Jolion,Ultra Hybrid Auto,2025 GWM Haval Jolion Ultra Hybrid Auto,"$39,485",Drive Away,Demo,14 km,5.1L / 100km,SUV,1.5L Petrol,Epsom
3,703627,KMHK381AMKU366359,[https://edge.pxcrush.net/cars/dealer/crrxmi57...,[Added 4 days ago],2019,Hyundai,Kona,Elite Auto 2WD MY19,2019 Hyundai Kona Elite Auto 2WD MY19,"$25,990",Drive Away,Used,"72,119 km",7.2L / 100km,SUV,2.0L Petrol,Epsom
4,507653,LGWEF6A51SH972803,[https://edge.pxcrush.net/cars/dealer/b6fwkfhq...,[],2025,GWM,Haval H6,Vanta Auto,2025 GWM Haval H6 Vanta Auto,"$36,990",Drive Away,New,13 km,7.4L / 100km,SUV,2.0L Petrol,Epsom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,507265,KMHJC81BWSU377542,[https://media.adtorqueedge.com/stock/images/j...,[],2025,Hyundai,Tucson,Premium Auto AWD MY25,2025 Hyundai Tucson Premium Auto AWD MY25,"$57,500",Drive Away,New,11 km,7.2L / 100km,SUV,1.6L Petrol,Epsom
192,507263,KMHJB811MSU332478,[https://media.adtorqueedge.com/stock/images/j...,[On Special],2025,Hyundai,Tucson,Elite Auto 2WD MY25,2025 Hyundai Tucson Elite Auto 2WD MY25,"$50,000",Drive Away,New,8 km,5.3L / 100km,SUV,1.6L Petrol,Epsom
193,506948,3GCUD9EL3RG345545,[https://media.adtorqueedge.com/stock/images/j...,[],2024,Chevrolet,Silverado,1500 ZR2 W/Tech Pack Auto 4x4 MY24,2024 Chevrolet Silverado 1500 ZR2 W/Tech Pack ...,"$139,888",Drive Away,Demo,"4,685 km",12.2L / 100km,Ute,6.2L Petrol,Epsom
194,507321,WVGZZZA18SV050512,[https://media.adtorqueedge.com/stock/images/j...,[],2025,Volkswagen,T-Roc,110TSI Style D11 Auto MY25,2025 Volkswagen T-Roc 110TSI Style D11 Auto MY25,POA,Price on Application,New,12 km,6.3L / 100km,SUV,1.4L Petrol,Epsom
